In [1]:
!pip install transformers #Using HuggingFace for getting the Google Flan-T5-small model
!pip install sentencepiece #T5 tokenizer uses SentencePiece tokenizer
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00


In [30]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoConfig
from transformers.data.metrics.squad_metrics import compute_exact, compute_f1

from datasets import load_metric, load_dataset
import torch
import tqdm

In [3]:
model_name = "google/flan-t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [6]:
dataset = load_dataset('McGill-NLP/TopiOCQA')
train_dataset = dataset['train']
val_dataset = dataset['validation']

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [7]:
# Initialize metrics
metric = load_metric("accuracy")
f1_metric = load_metric("f1")

# Evaluate the model
model.eval()
# Initialize lists to store predictions and targets
predictions = []
references = []

<ipython-input-7-1f7b240132eb>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [15]:
c = val_dataset.__len__()

In [16]:
for example in tqdm.tqdm(val_dataset):
    input_text = example["Rationale"]
    question = example["Question"]
    reference = example["Answer"]

    input_text = f"question: {question} context: {input_text}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    inputs = {k: v for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(**inputs)

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    predictions.append(answer)
    references.append(reference)

  0%|          | 0/2514 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 2514/2514 [36:06<00:00,  1.16it/s]


In [31]:
# Initialize lists to store evaluation results
all_exact_scores = []
all_f1_scores = []

for i in range(len(val_dataset)):
    # Calculate exact match (EM) and F1 score
    exact_score = compute_exact(references[i], predictions[i])
    f1_score = compute_f1(references[i], predictions[i])

    all_exact_scores.append(exact_score)
    all_f1_scores.append(f1_score)

In [33]:
# Calculate average metrics
average_exact_score = sum(all_exact_scores) / len(all_exact_scores)
average_f1_score = sum(all_f1_scores) / len(all_f1_scores)

print(f"Average Exact Match (EM) score: {average_exact_score}")
print(f"Average F1 score: {average_f1_score}")

Average Exact Match (EM) score: 0.15672235481304694
Average F1 score: 0.3191207599427022
